In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *
import xgboost as xgb

from sklearn.preprocessing import PowerTransformer


import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 150)

In [2]:
df = pd.read_csv("EDAed_df.csv")

df["Policy Start Date"] = pd.to_datetime(df["Policy Start Date"])

In [3]:
df.shape

(2000000, 50)

In [4]:
df.isnull().sum()

Age                                 0
Gender                              0
Annual Income                       0
Marital Status                      0
Number of Dependents                0
Education Level                     0
Occupation                          0
Health Score                        0
Location                            0
Policy Type                         0
Previous Claims                     0
Vehicle Age                         0
Credit Score                        0
Insurance Duration                  0
Policy Start Date                   0
Customer Feedback                   0
Smoking Status                      0
Exercise Frequency                  0
Property Type                       0
Premium Amount                 800000
IsNull_Age                          0
IsNull_Annual Income                0
IsNull_Marital Status               0
IsNull_Number of Dependents         0
IsNull_Occupation                   0
IsNull_Health Score                 0
IsNull_Previ

In [5]:
train = df.iloc[:1200000, :]
train.shape

(1200000, 50)

In [6]:
test = df.iloc[1200000:, :]
test.shape

(800000, 50)

In [7]:
test.head(3)

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,IsNull_Age,IsNull_Annual Income,IsNull_Marital Status,IsNull_Number of Dependents,IsNull_Occupation,IsNull_Health Score,IsNull_Previous Claims,IsNull_Vehicle Age,IsNull_Credit Score,IsNull_Insurance Duration,IsNull_Customer Feedback,Health Conscious Level,Health Conscious Level1,Money Per Head,Money Handling Level,Money Handling Level1,Growth,Growth1,Determinstic,Day_Name,Credit by Score,CreditInsurance,Health_Risk_Score,Credit_Health_Score,Health_Age_Interaction,Feedback1,Feedback2,Feedback3,Feedback4,Total Nulls
1200000,28.0,Female,2310.0,Divorced,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,2.0,19.0,551.0,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House,NaN,0,0,1,0,0,0,1,0,1,0,0,2,3430.775431,577.5,1272810.0,4.192377,4620.0,1155.000000,82.500000,Sunday,275.5,551.0,4.617101,4219.547460,214.423464,4620.0,1102.0,4.0,15.315962,3
1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,1.0,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment,NaN,0,0,0,0,0,0,1,0,0,0,0,1,1659.291012,63015.5,46883532.0,338.793011,378093.0,42010.333333,4065.516129,Monday,372.0,2976.0,4.330931,4977.873036,414.822753,1008248.0,2976.0,8.0,107.051033,1
1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,1.0,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo,NaN,0,0,0,0,0,0,1,0,0,0,0,3,9157.302066,17092.0,13998348.0,20.869353,68368.0,4273.000000,363.659574,Wednesday,819.0,7371.0,3.782274,19946.357425,1144.662758,68368.0,3276.0,4.0,97.418107,1


#
---
#

# Adding Dates columns

In [8]:
df["Policy Start Date - Day"] = df["Policy Start Date"].dt.day
df["Policy Start Date - Month"] = df["Policy Start Date"].dt.month
df["Policy Start Date - Year"] = df["Policy Start Date"].dt.year

In [9]:
df["Policy Start Date - Quarter"] = df["Policy Start Date"].dt.year.astype(str) + " Q" + df["Policy Start Date"].dt.quarter.astype(str)

In [10]:
df["Sin_Date"] = np.sin(2 * np.pi * df["Policy Start Date"].astype('int64'))
df["Cos_Date"] = np.cos(2 * np.pi * df["Policy Start Date"].astype('int64'))

In [11]:
df["Sin_Year"] = np.sin(2 * np.pi * df["Policy Start Date - Year"].astype('int64'))
df["Cos_Year"] = np.cos(2 * np.pi * df["Policy Start Date - Year"].astype('int64'))

In [12]:
df["Sin_Month"] = np.sin(2 * np.pi * df["Policy Start Date - Month"].astype('int64'))
df["Cos_Month"] = np.cos(2 * np.pi * df["Policy Start Date - Month"].astype('int64'))

In [13]:
df.head(3)

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,IsNull_Age,IsNull_Annual Income,IsNull_Marital Status,IsNull_Number of Dependents,IsNull_Occupation,IsNull_Health Score,IsNull_Previous Claims,IsNull_Vehicle Age,IsNull_Credit Score,IsNull_Insurance Duration,IsNull_Customer Feedback,Health Conscious Level,Health Conscious Level1,Money Per Head,Money Handling Level,Money Handling Level1,Growth,Growth1,Determinstic,Day_Name,Credit by Score,CreditInsurance,Health_Risk_Score,Credit_Health_Score,Health_Age_Interaction,Feedback1,Feedback2,Feedback3,Feedback4,Total Nulls,Policy Start Date - Day,Policy Start Date - Month,Policy Start Date - Year,Policy Start Date - Quarter,Sin_Date,Cos_Date,Sin_Year,Cos_Year,Sin_Month,Cos_Month
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0,0,0,0,0,0,0,0,0,0,0,0,4,13740.046488,10049.000000,3738228.0,27.013441,20098.0,5024.500000,528.894737,Saturday,186.0,1860.0,3.870062,8406.738970,429.376453,20098.0,744.0,4.0,45.197521,0,23,12,2023,2023 Q4,-0.975344,-0.220691,-6.447061e-13,1.0,-2.939152e-15,1.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0,0,0,0,0,1,0,0,0,0,0,0,2,4857.756069,10559.333333,21984532.0,45.645533,95034.0,10559.333333,812.256410,Monday,694.0,1388.0,4.221513,10805.393307,607.219509,126712.0,2776.0,4.0,62.278924,1,12,6,2023,2023 Q2,-0.998725,0.050489,-6.447061e-13,1.0,-1.469576e-15,1.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,632.0,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0,0,0,0,0,0,0,0,0,1,0,0,4,17361.338138,8534.000000,16180464.0,40.509494,25602.0,25602.000000,1113.130435,Saturday,632.0,1896.0,2.641123,29816.211150,1085.083634,204816.0,5056.0,8.0,377.420394,1,30,9,2023,2023 Q3,-0.994867,0.101192,-6.447061e-13,1.0,-2.204364e-15,1.0


#
---
#

In [14]:
data = df.copy()

#
---
#

In [15]:
df.drop(columns="Policy Start Date", inplace=True)

In [16]:
df.head(3)

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,IsNull_Age,IsNull_Annual Income,IsNull_Marital Status,IsNull_Number of Dependents,IsNull_Occupation,IsNull_Health Score,IsNull_Previous Claims,IsNull_Vehicle Age,IsNull_Credit Score,IsNull_Insurance Duration,IsNull_Customer Feedback,Health Conscious Level,Health Conscious Level1,Money Per Head,Money Handling Level,Money Handling Level1,Growth,Growth1,Determinstic,Day_Name,Credit by Score,CreditInsurance,Health_Risk_Score,Credit_Health_Score,Health_Age_Interaction,Feedback1,Feedback2,Feedback3,Feedback4,Total Nulls,Policy Start Date - Day,Policy Start Date - Month,Policy Start Date - Year,Policy Start Date - Quarter,Sin_Date,Cos_Date,Sin_Year,Cos_Year,Sin_Month,Cos_Month
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,Poor,No,Weekly,House,2869.0,0,0,0,0,0,0,0,0,0,0,0,4,13740.046488,10049.000000,3738228.0,27.013441,20098.0,5024.500000,528.894737,Saturday,186.0,1860.0,3.870062,8406.738970,429.376453,20098.0,744.0,4.0,45.197521,0,23,12,2023,2023 Q4,-0.975344,-0.220691,-6.447061e-13,1.0,-2.939152e-15,1.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,Average,Yes,Monthly,House,1483.0,0,0,0,0,1,0,0,0,0,0,0,2,4857.756069,10559.333333,21984532.0,45.645533,95034.0,10559.333333,812.256410,Monday,694.0,1388.0,4.221513,10805.393307,607.219509,126712.0,2776.0,4.0,62.278924,1,12,6,2023,2023 Q2,-0.998725,0.050489,-6.447061e-13,1.0,-1.469576e-15,1.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,632.0,3.0,Good,Yes,Weekly,House,567.0,0,0,0,0,0,0,0,0,1,0,0,4,17361.338138,8534.000000,16180464.0,40.509494,25602.0,25602.000000,1113.130435,Saturday,632.0,1896.0,2.641123,29816.211150,1085.083634,204816.0,5056.0,8.0,377.420394,1,30,9,2023,2023 Q3,-0.994867,0.101192,-6.447061e-13,1.0,-2.204364e-15,1.0


In [17]:
df[["Policy Start Date - Day", "Policy Start Date - Month", "Policy Start Date - Year"]] = df[["Policy Start Date - Day", "Policy Start Date - Month", "Policy Start Date - Year"]].astype("O")

In [18]:
def show_nulls(df):
    nulls = []
    nuniques = []
    uniques = []
    types = []
    
    for i in df.columns:
        nulls.append(df[i].isnull().sum())
        nuniques.append(df[i].nunique())
        uniques.append(df[i].unique())
        types.append(df[i].dtype)
    
    
    return pd.DataFrame(
        {
            "Column" : df.columns,
            "Data Type" : types,
            "Nulls" : nulls,
            "No. of Uniques" : nuniques,
            "Uniques" : uniques
        }
    ).sort_values(by="Nulls", ascending=False)

In [19]:
df["Health Conscious Level"] = df["Health Conscious Level"].astype("O")

In [20]:
show_nulls(df)

,Column,Data Type,Nulls,No. of Uniques,Uniques
18,Premium Amount,float64,800000,4794,"[2869.0, 1483.0, 567.0, 765.0, 2022.0, 3202.0,..."
1,Gender,object,0,2,"[Female, Male]"
0,Age,float64,0,47,"[19.0, 39.0, 23.0, 21.0, 29.0, 41.0, 48.0, 44...."
3,Marital Status,object,0,3,"[Married, Divorced, Single]"
4,Number of Dependents,float64,0,5,"[1.0, 3.0, 2.0, 0.0, 4.0]"
5,Education Level,object,0,4,"[Bachelor's, Master's, High School, PhD]"
6,Occupation,object,0,3,"[Self-Employed, Unemployed, Employed]"
7,Health Score,float64,0,933976,"[22.59876067181393, 15.569730989408043, 47.177..."
8,Location,object,0,3,"[Urban, Rural, Suburban]"
9,Policy Type,object,0,3,"[Premium, Comprehensive, Basic]"


#
---
#

In [21]:
def do_magic(target_column, *columns: list):
    for i in columns:
        df[f"{i}_MIN_{target_column}"] = df.groupby(by=i)[target_column].transform("min")
        df[f"{i}_MEAN_{target_column}"] = df.groupby(by=i)[target_column].transform("mean")
        df[f"{i}_MEDIAN_{target_column}"] = df.groupby(by=i)[target_column].transform("median")
        df[f"{i}_STD_{target_column}"] = df.groupby(by=i)[target_column].transform("std")
        df[f"{i}_MAX_{target_column}"] = df.groupby(by=i)[target_column].transform("max")

In [22]:
do_magic("Premium Amount", "Number of Dependents", "Occupation", "Education Level", "Previous Claims", "Health Conscious Level", "Insurance Duration")

In [23]:
df.isnull().sum()

Age                                                  0
Gender                                               0
Annual Income                                        0
Marital Status                                       0
Number of Dependents                                 0
Education Level                                      0
Occupation                                           0
Health Score                                         0
Location                                             0
Policy Type                                          0
Previous Claims                                      0
Vehicle Age                                          0
Credit Score                                         0
Insurance Duration                                   0
Customer Feedback                                    0
Smoking Status                                       0
Exercise Frequency                                   0
Property Type                                        0
Premium Am

#
---
#

In [24]:
def return_splits(ddf, feature_name, target_name):
    return [ddf[ddf[feature_name] == i][target_name] for i in ddf[feature_name].unique()]

def give_stats_analysis(df, target_column_name):
    ddf = df.copy()
    ddf = ddf.dropna()

    features = []
    tests = []
    stats = []
    pvals = []
    verdict = []
    count = 0

    target = ddf[target_column_name]
    for i in ddf.columns:
        features.append(i)
        feature = ddf[i]
        
        if (feature.dtype == "O" and (target.dtype == "float" or target.dtype == "int")) or (target.dtype == "O" and (feature.dtype == "float" or feature.dtype == "int")):
            stat, pval, *_ = kruskal(*return_splits(ddf, feature.name, target.name))
            tests.append("Kruskal-Wallis")
            stats.append(stat)
            pvals.append(pval)
            
        
        elif (feature.dtype == "float" or feature.dtype == "int") and (target.dtype == "float" or target.dtype == "int"):
            stat, pval, *_ = spearmanr(feature, target)
            tests.append("SpearmanR")
            stats.append(stat)
            pvals.append(pval)

        elif feature.dtype == "O" and target.dtype == "O":
            stat, pval, *_ = chi2_contingency(pd.crosstab(feature, target))
            tests.append("Chi-Square")
            stats.append(stat)
            pvals.append(pval)
        
        else:
            tests.append(np.nan)
            stats.append(np.nan)
            pvals.append(np.nan)
        
        if pval <= 0.025:
            verdict.append("There is Relationship")
        else:
            verdict.append("There is NO Relationship")

        print(f"{feature.name} ■■■ {target_column_name}".ljust(100, "-")+"✅")
    
    return pd.DataFrame({
        "Feature" : features,
        "Target" : [target_column_name]*ddf.shape[1],
        "Statistic Test" : tests,
        "Test Statistic" : stats,
        "P-Value" : pvals,
        "Verdict" : verdict
    }).sort_values(by="P-Value")

# H0 :- There is ***No Relationship*** among the given two columns
# H1 :- There is ***Relationship*** among the given two columns

### ***Health-related indicators***
- [x] Health Score
- [x] Smoking Status
- [x] Exercise Frequency
### ***Demographic information***
- [x] Age
- [x] Gender
- [x] Marital Status
- [x] Number of Dependents
- [x] Occupation
### ***Policy details***
- [x] Policy Type
- [x] Policy Start Date
- [x] Insurance Duration
### ***Financial factors***
- [x] Annual Income
- [x] Credit Score.
### ***Premium calculation***
- [x] Premium Amount

In [25]:
stats_result = give_stats_analysis(df.iloc[:1200000, :], "Premium Amount")
stats_result

Age ■■■ Premium Amount------------------------------------------------------------------------------✅
Gender ■■■ Premium Amount---------------------------------------------------------------------------✅
Annual Income ■■■ Premium Amount--------------------------------------------------------------------✅
Marital Status ■■■ Premium Amount-------------------------------------------------------------------✅
Number of Dependents ■■■ Premium Amount-------------------------------------------------------------✅
Education Level ■■■ Premium Amount------------------------------------------------------------------✅
Occupation ■■■ Premium Amount-----------------------------------------------------------------------✅
Health Score ■■■ Premium Amount---------------------------------------------------------------------✅
Location ■■■ Premium Amount-------------------------------------------------------------------------✅
Policy Type ■■■ Premium Amount----------------------------------------------------

,Feature,Target,Statistic Test,Test Statistic,P-Value,Verdict
2,Annual Income,Premium Amount,SpearmanR,-0.061831,0.000000e+00,There is Relationship
12,Credit Score,Premium Amount,SpearmanR,-0.036687,0.000000e+00,There is Relationship
20,IsNull_Annual Income,Premium Amount,SpearmanR,-0.065399,0.000000e+00,There is Relationship
18,Premium Amount,Premium Amount,SpearmanR,1.000000,0.000000e+00,There is Relationship
35,Growth,Premium Amount,SpearmanR,-0.055000,0.000000e+00,There is Relationship
34,Money Handling Level1,Premium Amount,SpearmanR,-0.048668,0.000000e+00,There is Relationship
33,Money Handling Level,Premium Amount,SpearmanR,-0.072097,0.000000e+00,There is Relationship
32,Money Per Head,Premium Amount,SpearmanR,-0.053422,0.000000e+00,There is Relationship
44,Feedback1,Premium Amount,SpearmanR,-0.053714,0.000000e+00,There is Relationship
39,Credit by Score,Premium Amount,SpearmanR,-0.054850,0.000000e+00,There is Relationship


# <ins>Key Premium as per Reseach Papers and as per dataset.</ins>
### `Strikeoff features are said by research and dataset too. But unstrike ones are not impactful to determine premium amount as per dataset but as per research it should be. We need to find why like so in these features`

- ### ~~Age~~
- ### Gender
- ### ~~Health Score~~
- ### Smoking Status
- ### Exercise Frequency
- ### ~~Occupation~~
- ### Policy Type
- ### ~~Previous Claims~~
- ### ~~Annual Income~~
- ### Insurance Duration
- ### ~~Credit Score~~

#
---
#

In [26]:
cols = ["Gender", "Smoking Status", "Exercise Frequency", "Policy Type", "Insurance Duration"]

In [27]:
# fig, axs = plt.subplots(2, 3, figsize=(20, 8))
# for col, ax in zip(cols, axs.flatten()):
#     sns.boxplot(y=df["Premium Amount"], x=df[col], color="mediumblue", ax=ax)

In [28]:
useless_columns = stats_result[stats_result["P-Value"] >= 0.05]["Feature"]
useless_columns

70          Education Level_MEAN_Premium Amount
71        Education Level_MEDIAN_Premium Amount
8                                      Location
63      Number of Dependents_MAX_Premium Amount
5                               Education Level
4                          Number of Dependents
54                                     Cos_Date
11                                  Vehicle Age
82    Health Conscious Level_STD_Premium Amount
19                                   IsNull_Age
9                                   Policy Type
73           Education Level_MAX_Premium Amount
17                                Property Type
3                                Marital Status
16                           Exercise Frequency
15                               Smoking Status
38                                     Day_Name
26                           IsNull_Vehicle Age
72           Education Level_STD_Premium Amount
28                    IsNull_Insurance Duration
49                      Policy Start Dat

In [29]:
meaningless_df = df[useless_columns]
meaningless_df.head(3)

,Education Level_MEAN_Premium Amount,Education Level_MEDIAN_Premium Amount,Location,Number of Dependents_MAX_Premium Amount,Education Level,Number of Dependents,Cos_Date,Vehicle Age,Health Conscious Level_STD_Premium Amount,IsNull_Age,Policy Type,Education Level_MAX_Premium Amount,Property Type,Marital Status,Exercise Frequency,Smoking Status,Day_Name,IsNull_Vehicle Age,Education Level_STD_Premium Amount,IsNull_Insurance Duration,Policy Start Date - Day,Gender,Sin_Date,Insurance Duration
0,1102.698438,873.0,Urban,4994.0,Bachelor's,1.0,-0.220691,17.0,864.569091,0,Premium,4988.0,House,Married,Weekly,No,Saturday,0,864.866296,0,23,Female,-0.975344,5.0
1,1102.113989,871.0,Rural,4997.0,Master's,3.0,0.050489,12.0,865.103831,0,Comprehensive,4997.0,House,Divorced,Monthly,Yes,Monday,0,866.235322,0,12,Female,-0.998725,2.0
2,1104.787490,876.0,Suburban,4997.0,High School,3.0,0.101192,14.0,864.569091,0,Premium,4999.0,House,Divorced,Weekly,Yes,Saturday,0,865.951488,0,30,Male,-0.994867,3.0


In [30]:
# df = df[stats_result[stats_result["P-Value"] < 0.05]["Feature"]]
# df.head(3)

# Compressing Meaningless DF's information in a component using PCA

In [31]:
meaningless_df.head(3)

,Education Level_MEAN_Premium Amount,Education Level_MEDIAN_Premium Amount,Location,Number of Dependents_MAX_Premium Amount,Education Level,Number of Dependents,Cos_Date,Vehicle Age,Health Conscious Level_STD_Premium Amount,IsNull_Age,Policy Type,Education Level_MAX_Premium Amount,Property Type,Marital Status,Exercise Frequency,Smoking Status,Day_Name,IsNull_Vehicle Age,Education Level_STD_Premium Amount,IsNull_Insurance Duration,Policy Start Date - Day,Gender,Sin_Date,Insurance Duration
0,1102.698438,873.0,Urban,4994.0,Bachelor's,1.0,-0.220691,17.0,864.569091,0,Premium,4988.0,House,Married,Weekly,No,Saturday,0,864.866296,0,23,Female,-0.975344,5.0
1,1102.113989,871.0,Rural,4997.0,Master's,3.0,0.050489,12.0,865.103831,0,Comprehensive,4997.0,House,Divorced,Monthly,Yes,Monday,0,866.235322,0,12,Female,-0.998725,2.0
2,1104.787490,876.0,Suburban,4997.0,High School,3.0,0.101192,14.0,864.569091,0,Premium,4999.0,House,Divorced,Weekly,Yes,Saturday,0,865.951488,0,30,Male,-0.994867,3.0


## Encoding Columns

In [32]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

###
---
###

In [33]:
meaningless_df["Location"].unique()

array(['Urban', 'Rural', 'Suburban'], dtype=object)

In [34]:
a = OrdinalEncoder(categories=[['Rural', 'Suburban', 'Urban']])

b = pd.DataFrame({"ENCODED_Location" : a.fit_transform(meaningless_df[["Location"]]).flatten()})

meaningless_df = pd.concat([meaningless_df, b], axis=1)
meaningless_df.drop(columns="Location", inplace=True)

###
---
###

In [35]:
meaningless_df["Education Level"].unique()

array(["Bachelor's", "Master's", 'High School', 'PhD'], dtype=object)

In [36]:
a = OrdinalEncoder(categories=[['High School', "Bachelor's", "Master's", 'PhD']])

b = pd.DataFrame({"ENCODED_Education Level" : a.fit_transform(meaningless_df[["Education Level"]]).flatten()})

meaningless_df = pd.concat([meaningless_df, b], axis=1)
meaningless_df.drop(columns="Education Level", inplace=True)

###
---
###

In [37]:
meaningless_df["Policy Type"].unique()

array(['Premium', 'Comprehensive', 'Basic'], dtype=object)

In [38]:
a = OrdinalEncoder(categories=[['Basic', 'Comprehensive', 'Premium']])

b = pd.DataFrame({"ENCODED_Policy Type" : a.fit_transform(meaningless_df[["Policy Type"]]).flatten()})

meaningless_df = pd.concat([meaningless_df, b], axis=1)
meaningless_df.drop(columns="Policy Type", inplace=True)

###
---
###

In [39]:
a = OneHotEncoder(drop="first", sparse_output=False)

b = pd.DataFrame(
        a.fit_transform(meaningless_df[["Property Type"]]),
        columns=a.get_feature_names_out()
    )

meaningless_df = pd.concat([meaningless_df, b], axis=1)
meaningless_df.drop(columns="Property Type", inplace=True)

###
---
###

In [40]:
meaningless_df["Exercise Frequency"].unique()

array(['Weekly', 'Monthly', 'Daily', 'Rarely'], dtype=object)

In [41]:
a = OrdinalEncoder(categories=[['Rarely', 'Monthly', 'Weekly', 'Daily']])

b = pd.DataFrame({"ENCODED_Exercise Frequency" : a.fit_transform(meaningless_df[["Exercise Frequency"]]).flatten()})

meaningless_df = pd.concat([meaningless_df, b], axis=1)
meaningless_df.drop(columns="Exercise Frequency", inplace=True)

###
---
###

In [42]:
a = OneHotEncoder(drop="first", sparse_output=False)

b = pd.DataFrame(
        a.fit_transform(meaningless_df[["Smoking Status"]]),
        columns=a.get_feature_names_out()
    )

meaningless_df = pd.concat([meaningless_df, b], axis=1)
meaningless_df.drop(columns="Smoking Status", inplace=True)

###
---
###

In [43]:
a = OneHotEncoder(drop="first", sparse_output=False)

b = pd.DataFrame(
        a.fit_transform(meaningless_df[["Gender"]]),
        columns=a.get_feature_names_out()
    )

meaningless_df = pd.concat([meaningless_df, b], axis=1)
meaningless_df.drop(columns="Gender", inplace=True)

###
---
###

In [44]:
meaningless_df["Policy Start Date - Day"] = meaningless_df["Policy Start Date - Day"].astype(int)

#
---
#

In [45]:
meaningless_df.head(3)

,Education Level_MEAN_Premium Amount,Education Level_MEDIAN_Premium Amount,Number of Dependents_MAX_Premium Amount,Number of Dependents,Cos_Date,Vehicle Age,Health Conscious Level_STD_Premium Amount,IsNull_Age,Education Level_MAX_Premium Amount,Marital Status,Day_Name,IsNull_Vehicle Age,Education Level_STD_Premium Amount,IsNull_Insurance Duration,Policy Start Date - Day,Sin_Date,Insurance Duration,ENCODED_Location,ENCODED_Education Level,ENCODED_Policy Type,Property Type_Condo,Property Type_House,ENCODED_Exercise Frequency,Smoking Status_Yes,Gender_Male
0,1102.698438,873.0,4994.0,1.0,-0.220691,17.0,864.569091,0,4988.0,Married,Saturday,0,864.866296,0,23,-0.975344,5.0,2.0,1.0,2.0,0.0,1.0,2.0,0.0,0.0
1,1102.113989,871.0,4997.0,3.0,0.050489,12.0,865.103831,0,4997.0,Divorced,Monday,0,866.235322,0,12,-0.998725,2.0,0.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0
2,1104.787490,876.0,4997.0,3.0,0.101192,14.0,864.569091,0,4999.0,Divorced,Saturday,0,865.951488,0,30,-0.994867,3.0,1.0,0.0,2.0,0.0,1.0,2.0,1.0,1.0


In [46]:
meaningless_df.dtypes

Education Level_MEAN_Premium Amount          float64
Education Level_MEDIAN_Premium Amount        float64
Number of Dependents_MAX_Premium Amount      float64
Number of Dependents                         float64
Cos_Date                                     float64
Vehicle Age                                  float64
Health Conscious Level_STD_Premium Amount    float64
IsNull_Age                                     int64
Education Level_MAX_Premium Amount           float64
Marital Status                                object
Day_Name                                      object
IsNull_Vehicle Age                             int64
Education Level_STD_Premium Amount           float64
IsNull_Insurance Duration                      int64
Policy Start Date - Day                        int64
Sin_Date                                     float64
Insurance Duration                           float64
ENCODED_Location                             float64
ENCODED_Education Level                      f

###
---
###

# Doing PCA on this `meaningless_df`

In [47]:
# from sklearn.decomposition import PCA

In [48]:
# pca = PCA(n_components=3)
# pca_df = pd.DataFrame(pca.fit_transform(meaningless_df), columns=['PC1_Meaningless_df', "PC2_Meaningless_df", "PC3_Meaningless_df"])
# pca_df

In [49]:
# pca.explained_variance_ratio_

###
---
###

# Combining 2 PCs of Meaningless_columns to the df

In [50]:
# df = pd.concat([df, pca_df.iloc[:, :2]], axis=1)

In [51]:
df.head()

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,IsNull_Age,IsNull_Annual Income,IsNull_Marital Status,IsNull_Number of Dependents,IsNull_Occupation,IsNull_Health Score,IsNull_Previous Claims,IsNull_Vehicle Age,IsNull_Credit Score,IsNull_Insurance Duration,IsNull_Customer Feedback,Health Conscious Level,Health Conscious Level1,Money Per Head,Money Handling Level,Money Handling Level1,Growth,Growth1,Determinstic,Day_Name,Credit by Score,CreditInsurance,Health_Risk_Score,Credit_Health_Score,Health_Age_Interaction,Feedback1,Feedback2,Feedback3,Feedback4,Total Nulls,Policy Start Date - Day,Policy Start Date - Month,Policy Start Date - Year,Policy Start Date - Quarter,Sin_Date,Cos_Date,Sin_Year,Cos_Year,Sin_Month,Cos_Month,Number of Dependents_MIN_Premium Amount,Number of Dependents_MEAN_Premium Amount,Number of Dependents_MEDIAN_Premium Amount,Number of Dependents_STD_Premium Amount,Number of Dependents_MAX_Premium Amount,Occupation_MIN_Premium Amount,Occupation_MEAN_Premium Amount,Occupation_MEDIAN_Premium Amount,Occupation_STD_Premium Amount,Occupation_MAX_Premium Amount,Education Level_MIN_Premium Amount,Education Level_MEAN_Premium Amount,Education Level_MEDIAN_Premium Amount,Education Level_STD_Premium Amount,Education Level_MAX_Premium Amount,Previous Claims_MIN_Premium Amount,Previous Claims_MEAN_Premium Amount,Previous Claims_MEDIAN_Premium Amount,Previous Claims_STD_Premium Amount,Previous Claims_MAX_Premium Amount,Health Conscious Level_MIN_Premium Amount,Health Conscious Level_MEAN_Premium Amount,Health Conscious Level_MEDIAN_Premium Amount,Health Conscious Level_STD_Premium Amount,Health Conscious Level_MAX_Premium Amount,Insurance Duration_MIN_Premium Amount,Insurance Duration_MEAN_Premium Amount,Insurance Duration_MEDIAN_Premium Amount,Insurance Duration_STD_Premium Amount,Insurance Duration_MAX_Premium Amount
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,Poor,No,Weekly,House,2869.0,0,0,0,0,0,0,0,0,0,0,0,4,13740.046488,10049.000000,3738228.0,27.013441,20098.0,5024.500000,528.894737,Saturday,186.0,1860.0,3.870062,8406.738970,429.376453,20098.0,744.0,4.0,45.197521,0,23,12,2023,2023 Q4,-0.975344,-0.220691,-6.447061e-13,1.0,-2.939152e-15,1.0,20.0,1104.678891,874.0,865.235996,4994.0,20.0,1100.430574,870.0,865.079864,4999.0,20.0,1102.698438,873.0,864.866296,4988.0,20.0,1151.583106,907.0,898.402950,4988.0,20.0,1102.677039,871.0,864.569091,4991.0,20.0,1100.812035,872.0,859.965806,4996.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,Unemployed,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,Average,Yes,Monthly,House,1483.0,0,0,0,0,1,0,0,0,0,0,0,2,4857.756069,10559.333333,21984532.0,45.645533,95034.0,10559.333333,812.256410,Monday,694.0,1388.0,4.221513,10805.393307,607.219509,126712.0,2776.0,4.0,62.278924,1,12,6,2023,2023 Q2,-0.998725,0.050489,-6.447061e-13,1.0,-1.469576e-15,1.0,20.0,1104.006551,875.0,864.955881,4997.0,20.0,1103.361209,872.0,867.023490,4997.0,20.0,1102.113989,871.0,866.235322,4997.0,20.0,1083.632645,855.0,853.156218,4997.0,20.0,1098.159650,862.0,865.103831,4999.0,20.0,1106.883166,878.0,863.675409,4997.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,632.0,3.0,Good,Yes,Weekly,House,567.0,0,0,0,0,0,0,0,0,1,0,0,4,17361.338138,8534.000000,16180464.0,40.509494,25602.0,25602.000000,1113.130435,Saturday,632.0,1896.0,2.641123,29816.211150,1085.083634,204816.0,5056.0,8.0,377.420394,1,30,9,2023,2023 Q3,-0.994867,0.101192,-6.447061e-13,1.0,-2.204364e-15,1.0,20.0,1104.006551,875.0,864.955881,4997.0,20.0,1100.430574,870.0,865.079864,4999.0,20.0,1104.787490,876.0,865.951488,4999.0,20.0,1083.632645,855.0,853.156218,4997.0,20.0,1102.677039,871.0,864.569091,4991.0,20.0,1101.733099,872.0,865.787949,4997.0
3,21.0,Mal

###
---
###

# Encoding of column in `df`

In [52]:
show_nulls(df)

,Column,Data Type,Nulls,No. of Uniques,Uniques
18,Premium Amount,float64,800000,4794,"[2869.0, 1483.0, 567.0, 765.0, 2022.0, 3202.0,..."
77,Previous Claims_STD_Premium Amount,float64,9,9,"[898.4029501785653, 853.1562175615868, 851.671..."
1,Gender,object,0,2,"[Female, Male]"
3,Marital Status,object,0,3,"[Married, Divorced, Single]"
2,Annual Income,float64,0,97970,"[10049.0, 31678.0, 25602.0, 141855.0, 39651.0,..."
5,Education Level,object,0,4,"[Bachelor's, Master's, High School, PhD]"
6,Occupation,object,0,3,"[Self-Employed, Unemployed, Employed]"
7,Health Score,float64,0,933976,"[22.59876067181393, 15.569730989408043, 47.177..."
4,Number of Dependents,float64,0,5,"[1.0, 3.0, 2.0, 0.0, 4.0]"
8,Location,object,0,3,"[Urban, Rural, Suburban]"


### Policy Start Date - Year	

In [53]:
df["Policy Start Date - Year"].unique()

array([2023, 2024, 2021, 2022, 2020, 2019], dtype=object)

In [54]:
a = OrdinalEncoder(categories=[[2019, 2020, 2021, 2022, 2023, 2024]])

b = pd.DataFrame({"ENCODED_Policy Start Date - Year" : a.fit_transform(df[["Policy Start Date - Year"]]).flatten()})

df = pd.concat([df, b], axis=1)
df.drop(columns="Policy Start Date - Year", inplace=True)

### Policy Start Date - Quarter

In [55]:
sorted(list(df["Policy Start Date - Quarter"].unique()))

['2019 Q3',
 '2019 Q4',
 '2020 Q1',
 '2020 Q2',
 '2020 Q3',
 '2020 Q4',
 '2021 Q1',
 '2021 Q2',
 '2021 Q3',
 '2021 Q4',
 '2022 Q1',
 '2022 Q2',
 '2022 Q3',
 '2022 Q4',
 '2023 Q1',
 '2023 Q2',
 '2023 Q3',
 '2023 Q4',
 '2024 Q1',
 '2024 Q2',
 '2024 Q3']

In [56]:
a = OrdinalEncoder(categories=[['2019 Q3', '2019 Q4', '2020 Q1', '2020 Q2', '2020 Q3', '2020 Q4', '2021 Q1', '2021 Q2', '2021 Q3',
                 '2021 Q4', '2022 Q1', '2022 Q2', '2022 Q3', '2022 Q4', '2023 Q1', '2023 Q2', '2023 Q3', '2023 Q4', '2024 Q1', '2024 Q2', '2024 Q3']])

b = pd.DataFrame({"ENCODED_Policy Start Date - Quarter" : a.fit_transform(df[["Policy Start Date - Quarter"]]).flatten()})

df = pd.concat([df, b], axis=1)
df.drop(columns="Policy Start Date - Quarter", inplace=True)

### Policy Start Date - Month

In [57]:
df["Policy Start Date - Month"] = df["Policy Start Date - Month"].astype(int)

### Customer Feedback

In [58]:
df["Customer Feedback"].unique()

array(['Poor', 'Average', 'Good'], dtype=object)

In [59]:
a = OrdinalEncoder(categories=[['Poor', 'Average', 'Good']])

b = pd.DataFrame({"ENCODED_Customer Feedback" : a.fit_transform(df[["Customer Feedback"]]).flatten()})

df = pd.concat([df, b], axis=1)
df.drop(columns="Customer Feedback", inplace=True)

### Occupation

In [60]:
a = OneHotEncoder(drop="first", sparse_output=False)

b = pd.DataFrame(
        a.fit_transform(df[["Occupation"]]),
        columns="ENCODED_" + a.get_feature_names_out()
    )

df = pd.concat([df, b], axis=1)
df.drop(columns="Occupation", inplace=True)

### Marital Status

In [61]:
a = OneHotEncoder(drop="first", sparse_output=False)

b = pd.DataFrame(
        a.fit_transform(df[["Marital Status"]]),
        columns="ENCODED_" + a.get_feature_names_out()
    )

df = pd.concat([df, b], axis=1)
df.drop(columns="Marital Status", inplace=True)

In [62]:
df

,Age,Gender,Annual Income,Number of Dependents,Education Level,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Smoking Status,Exercise Frequency,Property Type,Premium Amount,IsNull_Age,IsNull_Annual Income,IsNull_Marital Status,IsNull_Number of Dependents,IsNull_Occupation,IsNull_Health Score,IsNull_Previous Claims,IsNull_Vehicle Age,IsNull_Credit Score,IsNull_Insurance Duration,IsNull_Customer Feedback,Health Conscious Level,Health Conscious Level1,Money Per Head,Money Handling Level,Money Handling Level1,Growth,Growth1,Determinstic,Day_Name,Credit by Score,CreditInsurance,Health_Risk_Score,Credit_Health_Score,Health_Age_Interaction,Feedback1,Feedback2,Feedback3,Feedback4,Total Nulls,Policy Start Date - Day,Policy Start Date - Month,Sin_Date,Cos_Date,Sin_Year,Cos_Year,Sin_Month,Cos_Month,Number of Dependents_MIN_Premium Amount,Number of Dependents_MEAN_Premium Amount,Number of Dependents_MEDIAN_Premium Amount,Number of Dependents_STD_Premium Amount,Number of Dependents_MAX_Premium Amount,Occupation_MIN_Premium Amount,Occupation_MEAN_Premium Amount,Occupation_MEDIAN_Premium Amount,Occupation_STD_Premium Amount,Occupation_MAX_Premium Amount,Education Level_MIN_Premium Amount,Education Level_MEAN_Premium Amount,Education Level_MEDIAN_Premium Amount,Education Level_STD_Premium Amount,Education Level_MAX_Premium Amount,Previous Claims_MIN_Premium Amount,Previous Claims_MEAN_Premium Amount,Previous Claims_MEDIAN_Premium Amount,Previous Claims_STD_Premium Amount,Previous Claims_MAX_Premium Amount,Health Conscious Level_MIN_Premium Amount,Health Conscious Level_MEAN_Premium Amount,Health Conscious Level_MEDIAN_Premium Amount,Health Conscious Level_STD_Premium Amount,Health Conscious Level_MAX_Premium Amount,Insurance Duration_MIN_Premium Amount,Insurance Duration_MEAN_Premium Amount,Insurance Duration_MEDIAN_Premium Amount,Insurance Duration_STD_Premium Amount,Insurance Duration_MAX_Premium Amount,ENCODED_Policy Start Date - Year,ENCODED_Policy Start Date - Quarter,ENCODED_Customer Feedback,ENCODED_Occupation_Self-Employed,ENCODED_Occupation_Unemployed,ENCODED_Marital Status_Married,ENCODED_Marital Status_Single
0,19.0,Female,10049.0,1.0,Bachelor's,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,No,Weekly,House,2869.0,0,0,0,0,0,0,0,0,0,0,0,4,13740.046488,10049.000000,3738228.0,27.013441,20098.0,5024.500000,528.894737,Saturday,186.0,1860.0,3.870062,8406.738970,429.376453,20098.0,744.0,4.0,45.197521,0,23,12,-0.975344,-0.220691,-6.447061e-13,1.0,-2.939152e-15,1.0,20.0,1104.678891,874.0,865.235996,4994.0,20.0,1100.430574,870.0,865.079864,4999.0,20.0,1102.698438,873.0,864.866296,4988.0,20.0,1151.583106,907.0,898.402950,4988.0,20.0,1102.677039,871.0,864.569091,4991.0,20.0,1100.812035,872.0,859.965806,4996.0,4.0,17.0,0.0,1.0,0.0,1.0,0.0
1,39.0,Female,31678.0,3.0,Master's,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,Yes,Monthly,House,1483.0,0,0,0,0,1,0,0,0,0,0,0,2,4857.756069,10559.333333,21984532.0,45.645533,95034.0,10559.333333,812.256410,Monday,694.0,1388.0,4.221513,10805.393307,607.219509,126712.0,2776.0,4.0,62.278924,1,12,6,-0.998725,0.050489,-6.447061e-13,1.0,-1.469576e-15,1.0,20.0,1104.006551,875.0,864.955881,4997.0,20.0,1103.361209,872.0,867.023490,4997.0,20.0,1102.113989,871.0,866.235322,4997.0,20.0,1083.632645,855.0,853.156218,4997.0,20.0,1098.159650,862.0,865.103831,4999.0,20.0,1106.883166,878.0,863.675409,4997.0,4.0,15.0,1.0,0.0,1.0,0.0,0.0
2,23.0,Male,25602.0,3.0,High School,47.177549,Suburban,Premium,1.0,14.0,632.0,3.0,Yes,Weekly,House,567.0,0,0,0,0,0,0,0,0,1,0,0,4,17361.338138,8534.000000,16180464.0,40.509494,25602.0,25602.000000,1113.130435,Saturday,632.0,1896.0,2.641123,29816.211150,1085.083634,204816.0,5056.0,8.0,377.420394,1,30,9,-0.994867,0.101192,-6.447061e-13,1.0,-2.204364e-15,1.0,20.0,1104.006551,875.0,864.955881,4997.0,20.0,1100.430574,870.0,865.079864,4999.0,20.0,1104.787490,876.0,865.951488,4999.0,20.0,1083.632645,855.0,853.156218,4997.0,20.0,1102.677039,871.0,864.569091,

#
---
#

In [63]:
def return_splits(ddf, feature_name, target_name):
    return [ddf[ddf[feature_name] == i][target_name] for i in ddf[feature_name].unique()]

def give_stats_analysis(df, target_column_name):
    ddf = df.copy()
    ddf = ddf.dropna()

    features = []
    tests = []
    stats = []
    pvals = []
    verdict = []
    count = 0

    target = ddf[target_column_name]
    for i in ddf.columns:
        features.append(i)
        feature = ddf[i]
        
        if (feature.dtype == "O" and (target.dtype == "float" or target.dtype == "int")) or (target.dtype == "O" and (feature.dtype == "float" or feature.dtype == "int")):
            stat, pval, *_ = kruskal(*return_splits(ddf, feature.name, target.name))
            tests.append("Kruskal-Wallis")
            stats.append(stat)
            pvals.append(pval)
            
        
        elif (feature.dtype == "float" or feature.dtype == "int") and (target.dtype == "float" or target.dtype == "int"):
            stat, pval, *_ = spearmanr(feature, target)
            tests.append("SpearmanR")
            stats.append(stat)
            pvals.append(pval)

        elif feature.dtype == "O" and target.dtype == "O":
            stat, pval, *_ = chi2_contingency(pd.crosstab(feature, target))
            tests.append("Chi-Square")
            stats.append(stat)
            pvals.append(pval)
        
        else:
            tests.append(np.nan)
            stats.append(np.nan)
            pvals.append(np.nan)
        
        if pval <= 0.05:
            verdict.append("There is Relationship")
        else:
            verdict.append("There is NO Relationship")

        print(f"{feature.name} ■■■ {target_column_name}".ljust(50, "-")+"✅")
    
    return pd.DataFrame({
        "Feature" : features,
        "Target" : [target_column_name]*ddf.shape[1],
        "Statistic Test" : tests,
        "Test Statistic" : stats,
        "P-Value" : pvals,
        "Verdict" : verdict
    }).sort_values(by="P-Value")

In [64]:
stats_df = give_stats_analysis(df, "Premium Amount")
stats_df

Age ■■■ Premium Amount----------------------------✅
Gender ■■■ Premium Amount-------------------------✅
Annual Income ■■■ Premium Amount------------------✅
Number of Dependents ■■■ Premium Amount-----------✅
Education Level ■■■ Premium Amount----------------✅
Health Score ■■■ Premium Amount-------------------✅
Location ■■■ Premium Amount-----------------------✅
Policy Type ■■■ Premium Amount--------------------✅
Previous Claims ■■■ Premium Amount----------------✅
Vehicle Age ■■■ Premium Amount--------------------✅
Credit Score ■■■ Premium Amount-------------------✅
Insurance Duration ■■■ Premium Amount-------------✅
Smoking Status ■■■ Premium Amount-----------------✅
Exercise Frequency ■■■ Premium Amount-------------✅
Property Type ■■■ Premium Amount------------------✅
Premium Amount ■■■ Premium Amount-----------------✅
IsNull_Age ■■■ Premium Amount---------------------✅
IsNull_Annual Income ■■■ Premium Amount-----------✅
IsNull_Marital Status ■■■ Premium Amount----------✅
IsNull_Numbe

,Feature,Target,Statistic Test,Test Statistic,P-Value,Verdict
2,Annual Income,Premium Amount,SpearmanR,-0.061831,0.000000e+00,There is Relationship
10,Credit Score,Premium Amount,SpearmanR,-0.036687,0.000000e+00,There is Relationship
15,Premium Amount,Premium Amount,SpearmanR,1.000000,0.000000e+00,There is Relationship
17,IsNull_Annual Income,Premium Amount,SpearmanR,-0.065399,0.000000e+00,There is Relationship
30,Money Handling Level,Premium Amount,SpearmanR,-0.072097,0.000000e+00,There is Relationship
31,Money Handling Level1,Premium Amount,SpearmanR,-0.048668,0.000000e+00,There is Relationship
29,Money Per Head,Premium Amount,SpearmanR,-0.053422,0.000000e+00,There is Relationship
32,Growth,Premium Amount,SpearmanR,-0.055000,0.000000e+00,There is Relationship
36,Credit by Score,Premium Amount,SpearmanR,-0.054850,0.000000e+00,There is Relationship
34,Determinstic,Premium Amount,SpearmanR,-0.056869,0.000000e+00,There is Relationship


In [65]:
wanted_columns = stats_df[stats_df["P-Value"] <= 0.05]["Feature"]

In [66]:
df = df[wanted_columns]

In [67]:
df.select_dtypes(include='O')

,Health Conscious Level
0,4
1,2
2,4
3,3
4,3
...,...
1999995,4
1999996,5
1999997,2
1999998,4


In [68]:
# df['Health Conscious Level'] = df['Health Conscious Level'].astype(int)

In [69]:
# df['Policy Start Date - Day'] = df['Policy Start Date - Day'].astype(int)

In [70]:
# df.drop(columns='Day_Name', inplace=True)

### Location 

In [71]:
# a = OneHotEncoder(drop="first", sparse_output=False)

# b = pd.DataFrame(
#         a.fit_transform(df[["Location"]]),
#         columns="ENCODED_" + a.get_feature_names_out()
#     )

# df = pd.concat([df, b], axis=1)
# df.drop(columns="Location", inplace=True)

### Property Type 

In [72]:
# a = OneHotEncoder(drop="first", sparse_output=False)

# b = pd.DataFrame(
#         a.fit_transform(df[["Property Type"]]),
#         columns="ENCODED_" + a.get_feature_names_out()
#     )

# df = pd.concat([df, b], axis=1)
# df.drop(columns="Property Type", inplace=True)

### Gender

In [73]:
# a = OneHotEncoder(drop="first", sparse_output=False)

# b = pd.DataFrame(
#         a.fit_transform(df[["Gender"]]),
#         columns="ENCODED_" + a.get_feature_names_out()
#     )

# df = pd.concat([df, b], axis=1)
# df.drop(columns="Gender", inplace=True)

### Smoking Status

In [74]:
# a = OneHotEncoder(drop="first", sparse_output=False)

# b = pd.DataFrame(
#         a.fit_transform(df[["Smoking Status"]]),
#         columns="ENCODED_" + a.get_feature_names_out()
#     )

# df = pd.concat([df, b], axis=1)
# df.drop(columns="Smoking Status", inplace=True)

### Education Level 

In [75]:
# df['Education Level'].unique()

In [76]:
# a = OrdinalEncoder(categories=[["High School", "Bachelor's", "Master's", "PhD"]])

# b = pd.DataFrame({"ENCODED_Education Level" : a.fit_transform(df[["Education Level"]]).flatten()})

# df = pd.concat([df, b], axis=1)
# df.drop(columns="Education Level", inplace=True)

### Policy Type

In [77]:
# df['Policy Type'].unique()

In [78]:
# a = OrdinalEncoder(categories=[["Basic", "Comprehensive", "Premium"]])

# b = pd.DataFrame({"ENCODED_Policy Type" : a.fit_transform(df[["Policy Type"]]).flatten()})

# df = pd.concat([df, b], axis=1)
# df.drop(columns="Policy Type", inplace=True)

### Exercise Frequency

In [79]:
# df['Exercise Frequency'].unique()

In [80]:
# a = OrdinalEncoder(categories=[["Rarely", "Monthly", "Weekly", "Daily"]])

# b = pd.DataFrame({"ENCODED_Exercise Frequency" : a.fit_transform(df[["Exercise Frequency"]]).flatten()})

# df = pd.concat([df, b], axis=1)
# df.drop(columns="Exercise Frequency", inplace=True)

#
---
#

# Spliting Data

In [81]:
train = df.iloc[:1200000, :]
test = df.iloc[1200000:, :]

train.shape, test.shape

((1200000, 57), (800000, 57))

In [82]:
X = train.drop(columns="Premium Amount")
Y = train["Premium Amount"]

In [83]:
from sklearn.model_selection import train_test_split

In [84]:
x_train, x_validate, y_train, y_validate = train_test_split(X, Y, test_size=10000)

In [85]:
x_validate.shape

(10000, 56)

In [86]:
test.drop(columns="Premium Amount", inplace=True)

In [87]:
test.shape

(800000, 56)

In [88]:
test

,Annual Income,Credit Score,IsNull_Annual Income,Money Handling Level,Money Handling Level1,Money Per Head,Growth,Credit by Score,Determinstic,Growth1,Feedback1,Previous Claims_MEDIAN_Premium Amount,IsNull_Health Score,Previous Claims_MEAN_Premium Amount,Previous Claims,Previous Claims_STD_Premium Amount,IsNull_Customer Feedback,Previous Claims_MAX_Premium Amount,Feedback3,IsNull_Previous Claims,IsNull_Marital Status,Health Score,Health_Risk_Score,Feedback2,CreditInsurance,Sin_Year,IsNull_Credit Score,Health_Age_Interaction,Total Nulls,ENCODED_Policy Start Date - Year,ENCODED_Policy Start Date - Quarter,Feedback4,IsNull_Number of Dependents,IsNull_Occupation,Health Conscious Level1,Sin_Month,Policy Start Date - Month,Health Conscious Level,Health Conscious Level_MEAN_Premium Amount,Health Conscious Level_MEDIAN_Premium Amount,Number of Dependents_MEAN_Premium Amount,Number of Dependents_MEDIAN_Premium Amount,Number of Dependents_STD_Premium Amount,Insurance Duration_MEAN_Premium Amount,Insurance Duration_MEDIAN_Premium Amount,Health Conscious Level_MAX_Premium Amount,Credit_Health_Score,Occupation_MEDIAN_Premium Amount,Occupation_MEAN_Premium Amount,Occupation_MAX_Premium Amount,Previous Claims_MIN_Premium Amount,Insurance Duration_MAX_Premium Amount,ENCODED_Occupation_Self-Employed,Age,Insurance Duration_STD_Premium Amount,Occupation_STD_Premium Amount
1200000,2310.0,551.0,0,1272810.0,4.192377,577.500000,4620.0,275.5,82.500000,1155.000000,4620.0,907.0,0,1151.583106,2.0,898.402950,0,4988.0,4.0,1,1,7.657981,4.617101,1102.0,551.0,-6.447061e-13,1,214.423464,3,4.0,15.0,15.315962,0,0,3430.775431,-1.469576e-15,6,2,1098.159650,862.0,1096.464223,864.0,864.717484,1097.042977,861.0,4999.0,4219.547460,870.0,1100.430574,4999.0,20.0,4988.0,1.0,28.0,865.431191,865.079864
1200001,126031.0,372.0,0,46883532.0,338.793011,63015.500000,378093.0,372.0,4065.516129,42010.333333,1008248.0,855.0,0,1083.632645,1.0,853.156218,0,4997.0,8.0,1,0,13.381379,4.330931,2976.0,2976.0,1.585375e-14,0,414.822753,1,5.0,19.0,107.051033,0,0,1659.291012,-9.797174e-16,4,1,1098.453623,871.0,1108.443461,876.0,866.852628,1105.876809,876.0,4981.0,4977.873036,870.0,1100.430574,4999.0,20.0,4994.0,1.0,31.0,868.009584,865.079864
1200002,17092.0,819.0,0,13998348.0,20.869353,17092.000000,68368.0,819.0,363.659574,4273.000000,68368.0,855.0,0,1083.632645,1.0,853.156218,0,4997.0,4.0,1,0,24.354527,3.782274,3276.0,7371.0,-6.447061e-13,0,1144.662758,1,4.0,15.0,97.418107,0,0,9157.302066,-9.797174e-16,4,3,1099.009424,867.0,1097.649985,867.0,862.779627,1095.676958,861.0,4997.0,19946.357425,872.0,1103.361209,4997.0,20.0,4988.0,0.0,47.0,862.910692,867.023490
1200003,30424.0,770.0,0,23426480.0,39.511688,10141.333333,121696.0,770.0,1086.571429,7606.000000,60848.0,855.0,0,1083.632645,1.0,853.156218,0,4997.0,2.0,0,0,5.136225,4.743189,1540.0,3850.0,-6.447061e-13,0,143.814305,0,4.0,17.0,10.272450,0,0,4602.057755,-2.449294e-15,10,3,1099.009424,867.0,1104.006551,875.0,864.955881,1100.812035,872.0,4997.0,3954.893383,870.0,1100.430574,4999.0,20.0,4996.0,1.0,28.0,859.965806,865.079864
1200004,10863.0,755.0,0,8201565.0,14.388079,5431.500000,10863.0,755.0,452.625000,10863.000000,43452.0,855.0,0,1083.632645,1.0,853.156218,0,4997.0,4.0,1,0,11.844155,4.407792,3020.0,5285.0,-1.468363e-13,0,284.259727,1,2.0,9.0,47.376621,0,0,9096.311250,-9.799650e-15,11,3,1099.009424,867.0,1108.443461,876.0,866.852628,1105.964863,877.0,4997.0,8942.337231,872.0,1103.361209,4997.0,20.0,4988.0,0.0,24.0,866.225414,867.023490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,38782.0,309.0,0,11983638.0,125.508091,38782.000000,77564.0,309.0,775.640000,19391.000000,155128.0,855.0,0,1082.452746,0.0,851.671355,0,4999.0,0.0,1,0,14.498639,4.275068,1236.0,618.0,-1.468363e-13,0,724.931945,2,2.0,8.0,57.994556,0,1,23197.822227,-1.714506e-15,7,4,1102.677039,87

##
---
##

# Scaling on `df` 

In [89]:
# fig, axs = plt.subplots(3, 6, figsize=(20, 9))

# for i, ax in zip(x_train.columns, axs.flatten()):
#     sns.kdeplot(x_train[i], ax=ax, color="darkgray", fill=True)

# plt.tight_layout()
# plt.show()

In [90]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

In [91]:
def do_scaling(scaler):
    var_cols = list(map(lambda x : x.replace(" ", "_"), x_train.columns))
    scalers = {}
    cols = x_train.select_dtypes("number").columns
    
    for i in range(len(cols)):
        scalers[f"SCALER_{var_cols[i]}"] = scaler
        
        x_train[f"SCALER_{var_cols[i]}"] = scalers[f"SCALER_{var_cols[i]}"].fit_transform(x_train[[cols[i]]]).flatten()
        x_train.drop(columns=cols[i], inplace=True)

        x_validate[f"SCALER_{var_cols[i]}"] = scalers[f"SCALER_{var_cols[i]}"].transform(x_validate[[cols[i]]]).flatten()
        x_validate.drop(columns=cols[i], inplace=True)

        test[f"SCALER_{var_cols[i]}"] = scalers[f"SCALER_{var_cols[i]}"].transform(test[[cols[i]]]).flatten()
        test.drop(columns=cols[i], inplace=True)
    
    return scalers

In [92]:
scaler_objects = do_scaling(RobustScaler())
scaler_objects

{'SCALER_Annual_Income': RobustScaler(),
 'SCALER_Credit_Score': RobustScaler(),
 'SCALER_IsNull_Annual_Income': RobustScaler(),
 'SCALER_Money_Handling_Level': RobustScaler(),
 'SCALER_Money_Handling_Level1': RobustScaler(),
 'SCALER_Money_Per_Head': RobustScaler(),
 'SCALER_Growth': RobustScaler(),
 'SCALER_Credit_by_Score': RobustScaler(),
 'SCALER_Determinstic': RobustScaler(),
 'SCALER_Growth1': RobustScaler(),
 'SCALER_Feedback1': RobustScaler(),
 'SCALER_Previous_Claims_MEDIAN_Premium_Amount': RobustScaler(),
 'SCALER_IsNull_Health_Score': RobustScaler(),
 'SCALER_Previous_Claims_MEAN_Premium_Amount': RobustScaler(),
 'SCALER_Previous_Claims': RobustScaler(),
 'SCALER_Previous_Claims_STD_Premium_Amount': RobustScaler(),
 'SCALER_IsNull_Customer_Feedback': RobustScaler(),
 'SCALER_Previous_Claims_MAX_Premium_Amount': RobustScaler(),
 'SCALER_Feedback3': RobustScaler(),
 'SCALER_IsNull_Previous_Claims': RobustScaler(),
 'SCALER_IsNull_Marital_Status': RobustScaler(),
 'SCALER_Heal

In [93]:
x_train.head(3)

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_Total_Nulls,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount
350200,3,0.110042,0.909871,0.0,0.428916,-0.052332,-0.095740,0.064663,-0.375405,0.693237,0.243080,0.831379,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,1.50,0.0,0.0,1.404595,-1.404595,1.817860,1.688972,0.0,0.0,0.184854,-0.5,0.333333,0.5,2.793057,0.0,0.0,-0.222862,4.898587e-16,-0.4,-0.976617,-0.8,0.000000,0.125,0.000000,-1.753579,-2.75,0.5,2.320886,0.000000,0.000000,0.0,0.0,-0.666667,0.0,-0.826087,-1.064823,0.469401
329291,5,-0.281239,0.072961,0.0,-0.261131,-0.238875,0.109543,-0.089212,0.307443,-0.332644,-0.275320,-0.356764,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.25,1.0,0.0,-0.298462,0.298462,-0.508400,-0.812336,0.0,0.0,0.139124,0.0,0.333333,0.5,-0.479856,0.0,0.0,0.115153,4.898587e-16,-0.4,0.000000,0.2,-0.904347,-0.875,-4.197114,-1.483870,-2.75,0.0,-0.181398,0.666667,0.380456,-0.6,0.0,-0.666667,0.0,0.565217,-0.132030,-0.530599
665504,0,0.989199,0.712446,0.0,1.549462,0.624480,0.830998,1.384713,-0.449838,2.030522,0.569841,0.921576,52.0,0.0,57.590036,1.0,30.472002,1.0,-4.5,0.50,0.0,0.0,-0.901925,0.901925,0.300619,1.247850,0.0,0.0,-0.869669,0.5,0.333333,0.7,-0.513541,0.0,1.0,-0.552722,-9.797174e-16,0.8,2.568729,2.2,0.631238,0.250,3.658057,0.260301,1.00,-2.0,-0.647962,-0.333333,-0.619544,0.4,0.0,0.000000,1.0,-0.782609,0.822189,0.000000


In [94]:
x_validate.head(3)

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_Total_Nulls,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount
384147,5,-0.446260,0.227468,0.0,-0.426137,-0.400106,-0.159276,-0.280664,-0.632686,-0.306365,-0.385100,-0.332896,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,0.5,0.0,0.0,1.093839,-1.093839,0.100796,-0.554766,-0.246291,0.0,0.355565,-0.5,-0.666667,-0.5,0.777294,0.0,0.0,1.059900,-9.797174e-16,0.8,0.000000,0.2,0.095653,0.000,0.540228,0.458999,1.50,0.000000,1.304500,0.666667,0.380456,-0.6,0.0,0.333333,0.0,-0.565217,-0.781814,-0.530599
140558,2,1.337687,-0.935622,0.0,0.666651,2.356675,0.570868,1.014182,-1.071197,2.328758,1.468119,1.226125,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,0.5,0.0,0.0,1.085090,-1.085090,-0.378426,-0.471776,-0.246291,0.0,0.193875,-0.5,-0.666667,-0.6,0.771635,0.0,0.0,-0.403441,-7.347881e-16,0.6,-1.208313,-1.8,0.000000,0.125,0.000000,0.032506,0.00,0.833333,0.240421,-0.333333,-0.619544,0.4,0.0,-0.333333,1.0,-0.695652,0.218186,0.000000
1198950,7,1.973507,-0.562232,0.0,1.496880,2.571628,1.632712,2.526678,-0.930421,1.066342,1.224652,0.628466,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,0.0,1.0,0.0,0.593596,-0.593596,-0.639257,0.184299,-0.246291,0.0,1.317607,1.0,-0.666667,-0.7,-0.191370,1.0,1.0,2.140172,4.898587e-16,-0.4,4.461773,3.8,0.631238,0.250,3.658057,0.277687,1.25,-1.000000,0.218182,0.666667,0.380456,-0.6,0.0,-0.666667,0.0,0.739130,0.161898,-0.530599


In [95]:
test.head(3)

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_Total_Nulls,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount
1200000,2,-0.602783,-0.210300,0.0,-0.617038,-0.524675,-0.491370,-0.486669,-0.797735,-0.510110,-0.468231,-0.497266,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,0.0,1.0,1.0,-0.967950,0.967950,-0.566755,-0.837009,0.0,1.0,-0.856072,1.0,0.333333,0.5,-0.696364,0.0,0.0,-0.443597,0.000000e+00,0.0,-1.208313,-1.8,-1.073045,-1.250,-0.459772,-1.483870,-2.75,0.833333,-0.873037,-0.333333,-0.619544,0.4,0.0,-0.666667,1.0,-0.565217,-0.132030,0.000000
1200001,1,2.778011,-0.978541,0.0,1.572877,4.610353,2.287976,3.460039,-0.485437,3.456819,1.759849,5.494543,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.5,1.0,0.0,-0.644130,0.644130,0.261715,0.069533,1.0,0.0,-0.618106,0.0,0.666667,0.9,0.142877,0.0,0.0,-0.517413,4.898587e-16,-0.4,-1.128160,0.0,0.631238,0.250,3.658057,0.260301,1.00,-2.166667,-0.805849,-0.333333,-0.619544,0.4,0.0,0.000000,1.0,-0.434783,0.822189,0.000000
1200002,3,-0.198851,0.939914,0.0,-0.006045,-0.268738,0.243751,0.186993,0.961165,-0.230086,-0.298188,-0.116681,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,-0.023288,0.023288,0.394341,1.712523,0.0,0.0,0.248552,0.0,0.333333,0.5,0.054750,0.0,0.0,-0.204981,4.898587e-16,-0.4,-0.976617,-0.8,-0.904347,-0.875,-4.197114,-1.753579,-2.75,0.500000,0.520349,0.000000,0.000000,0.0,0.0,-0.666667,0.0,0.260870,-1.064823,0.469401


#
---
#

# Joining All Data

In [96]:
train = pd.concat([pd.concat([x_train, y_train], axis=1), pd.concat([x_validate, y_validate], axis=1)]).sort_index()
train.head(3)

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_Total_Nulls,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount,Premium Amount
0,4,-0.391308,-0.978541,0.0,-0.498665,-0.174446,-0.069759,-0.323104,-1.087379,-0.065518,-0.257205,-0.404860,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,0.0,0.0,0.0,-0.122626,0.122626,-0.725022,-0.347664,0.0,0.0,-0.600824,-0.5,0.333333,0.7,-0.422991,0.0,0.0,-0.014024,-1.469576e-15,1.2,0.023383,0.0,0.095653,0.000,0.540228,-0.739699,0.0,-0.500000,-0.502054,-0.333333,-0.619544,0.4,0.0,0.222222,1.0,-0.956522,-2.154674,0.000000,2869.0
1,2,0.199725,0.403433,0.0,0.377397,0.111496,-0.047042,0.468789,0.556634,0.216699,0.044642,0.231642,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,-0.520317,0.520317,0.173298,-0.524112,0.0,0.0,-0.389642,0.0,0.333333,0.5,-0.266722,0.0,1.0,-0.384137,0.000000e+00,0.0,-1.208313,-1.8,0.000000,0.125,0.000000,0.458999,1.5,0.833333,-0.289535,0.000000,0.000000,0.0,0.0,0.333333,0.0,-0.086957,-0.781814,0.469401,1483.0
2,4,0.033693,0.137339,0.0,0.098726,0.032674,-0.137197,-0.264940,0.355987,0.516358,0.865007,0.697934,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.5,0.0,0.0,1.267999,-1.267999,1.181256,-0.334206,0.0,1.0,0.177804,0.0,0.333333,0.6,2.616359,0.0,0.0,0.136870,-7.347881e-16,0.6,0.023383,0.0,0.000000,0.125,0.000000,-0.557842,0.0,-0.500000,1.394813,-0.333333,-0.619544,0.4,0.0,0.333333,1.0,-0.782609,0.000000,0.000000,567.0


In [97]:
test.head(3)

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_Total_Nulls,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount
1200000,2,-0.602783,-0.210300,0.0,-0.617038,-0.524675,-0.491370,-0.486669,-0.797735,-0.510110,-0.468231,-0.497266,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,0.0,1.0,1.0,-0.967950,0.967950,-0.566755,-0.837009,0.0,1.0,-0.856072,1.0,0.333333,0.5,-0.696364,0.0,0.0,-0.443597,0.000000e+00,0.0,-1.208313,-1.8,-1.073045,-1.250,-0.459772,-1.483870,-2.75,0.833333,-0.873037,-0.333333,-0.619544,0.4,0.0,-0.666667,1.0,-0.565217,-0.132030,0.000000
1200001,1,2.778011,-0.978541,0.0,1.572877,4.610353,2.287976,3.460039,-0.485437,3.456819,1.759849,5.494543,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.5,1.0,0.0,-0.644130,0.644130,0.261715,0.069533,1.0,0.0,-0.618106,0.0,0.666667,0.9,0.142877,0.0,0.0,-0.517413,4.898587e-16,-0.4,-1.128160,0.0,0.631238,0.250,3.658057,0.260301,1.00,-2.166667,-0.805849,-0.333333,-0.619544,0.4,0.0,0.000000,1.0,-0.434783,0.822189,0.000000
1200002,3,-0.198851,0.939914,0.0,-0.006045,-0.268738,0.243751,0.186993,0.961165,-0.230086,-0.298188,-0.116681,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,-0.023288,0.023288,0.394341,1.712523,0.0,0.0,0.248552,0.0,0.333333,0.5,0.054750,0.0,0.0,-0.204981,4.898587e-16,-0.4,-0.976617,-0.8,-0.904347,-0.875,-4.197114,-1.753579,-2.75,0.500000,0.520349,0.000000,0.000000,0.0,0.0,-0.666667,0.0,0.260870,-1.064823,0.469401


In [98]:
df = pd.concat([train, test])

#
---
#

# Download the `Model Ready df`

In [99]:
df.to_csv("trainable_df.csv", index=False)